In [1]:
library(survey)
library(tidyverse)

Loading required package: grid
Loading required package: Matrix
Loading required package: survival

Attaching package: ‘survey’

The following object is masked from ‘package:graphics’:

    dotchart

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.3.4     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_ph_flag,lab_ph_first,lab_ph_min,lab_ph_max,lab_ph_abnormal,sedative,echo_int,mort_28_day_int,ps,ps_weight
201220,125078,66690,0,2106-04-27 01:47:50,2106-05-01 11:25:46,1,62.67646,1,NA,⋯,1,7.45,7.42,7.45,1,1,0,1,0.4447784,1.801083
215842,151232,11663,0,2188-02-14 01:48:15,2188-02-15 19:02:48,1,86.76186,1,NA,⋯,0,NA,NA,NA,NA,0,0,0,0.1580691,1.187746
234312,164444,86645,1,2165-06-22 01:47:16,2165-07-07 13:55:20,1,56.08904,1,NA,⋯,1,7.49,7.49,7.49,1,0,0,0,0.3088600,1.446885
289157,146726,10304,0,2156-06-23 14:26:00,2156-06-30 09:26:00,1,45.91093,1,2156-06-25,⋯,1,7.44,7.44,7.44,1,0,1,0,0.7000591,1.428451
211964,160170,94534,0,2160-03-05 14:23:19,2160-03-06 22:48:41,1,59.38693,1,NA,⋯,0,NA,NA,NA,NA,0,0,0,0.3324814,1.498086
230173,112553,31544,0,2140-01-30 20:39:25,2140-02-02 18:41:39,1,300.00345,1,NA,⋯,0,NA,NA,NA,NA,0,0,1,0.2379659,1.312277


In [4]:
table_one <- data.table::fread(file.path(data_dir, "tableone_raw.csv"), data.table = FALSE)
head(table_one)

p_val,wtd_p_val,main_0,main_1,sub_0,sub_1,wtd_main_0,wtd_main_1,wtd_sub_0,wtd_sub_1,var
1.942281e-03,6.571404e-03,84.183688,79.166557,67.137454,59.720158,80.056488,75.755236,62.335234,55.313341,age
1.416420e-02,1.507728e-02,50.822846,47.665687,NA,NA,47.089492,43.841849,NA,NA,female
6.415559e-11,1.909584e-13,78.562949,82.979049,23.579777,26.702495,85.637556,91.568834,27.685863,32.234984,weight
1.705518e-15,4.041212e-14,8.505970,10.045054,7.451124,7.681609,8.332934,9.888314,7.564772,7.744827,elix_score
1.378591e-15,8.274714e-12,19.632462,20.778975,5.788347,5.446293,19.793740,20.807581,5.667359,5.364432,saps
7.217556e-27,0.000000e+00,5.304937,6.323865,3.618100,3.796365,5.528346,6.510264,3.764854,3.812443,sofa


In [5]:
wpv <- table_one %>%
    filter(wtd_p_val < 0.05) %>%
    pull(var) %>%
    gsub("female", "gender", .)

if (length(grep("day", wpv)) > 0) wpv <- wpv %>% discard(grepl("day", .)) %>% c(., "icu_adm_weekday")

wpv

[1] "age"                        "gender"                    
 [3] "weight"                     "elix_score"                
 [5] "saps"                       "sofa"                      
 [7] "vent"                       "vaso"                      
 [9] "sedative"                   "icd_chf"                   
[11] "icd_afib"                   "icd_renal"                 
[13] "icd_copd"                   "icd_cad"                   
[15] "icd_stroke"                 "icd_malignancy"            
[17] "vs_heart_rate_first"        "vs_temp_first"             
[19] "lab_platelet_first"         "lab_sodium_first"          
[21] "lab_potassium_first"        "lab_chloride_first"        
[23] "lab_bun_first"              "lab_creatinine_first"      
[25] "lab_po2_first"              "lab_pco2_first"            
[27] "lab_bnp_flag"               "lab_troponin_flag"         
[29] "lab_creatinine_kinase_flag" "icu_adm_weekday"

In [6]:
design_echo_ps_ate <- svydesign(ids = ~ icustay_id, weights = ~ ps_weight, data = full_data)

In [7]:
fml <- wpv %>%
c("echo", .) %>%
paste(collapse = " + ") %>%
sprintf("mort_28_day ~ %s", .)

fml

[1] "mort_28_day ~ echo + age + gender + weight + elix_score + saps + sofa + vent + vaso + sedative + icd_chf + icd_afib + icd_renal + icd_copd + icd_cad + icd_stroke + icd_malignancy + vs_heart_rate_first + vs_temp_first + lab_platelet_first + lab_sodium_first + lab_potassium_first + lab_chloride_first + lab_bun_first + lab_creatinine_first + lab_po2_first + lab_pco2_first + lab_bnp_flag + lab_troponin_flag + lab_creatinine_kinase_flag + icu_adm_weekday"

In [8]:
logi <- svyglm(as.formula(fml), family = quasibinomial, design = design_echo_ps_ate)

In [9]:
summary(logi)


Call:
svyglm(formula = as.formula(fml), family = quasibinomial, design = design_echo_ps_ate)

Survey design:
svydesign(ids = ~icustay_id, weights = ~ps_weight, data = full_data)

Coefficients:
                              Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 -1.5813112  1.7682963  -0.894 0.371244    
echo1                       -0.3147708  0.0922149  -3.413 0.000649 ***
age                          0.0032980  0.0007506   4.394 1.15e-05 ***
gender1                      0.1341377  0.0938740   1.429 0.153116    
weight                      -0.0099803  0.0021078  -4.735 2.28e-06 ***
elix_score                   0.0317635  0.0067661   4.695 2.77e-06 ***
saps                         0.0719846  0.0114975   6.261 4.28e-10 ***
sofa                         0.1723286  0.0172055  10.016  < 2e-16 ***
vent1                        0.1584679  0.1558667   1.017 0.309370    
vaso1                        0.0393363  0.1068834   0.368 0.712872    
sedative1                

In [10]:
exp(cbind(OR = coef(logi), confint(logi)))

,OR,2.5 %,97.5 %
(Intercept),0.2057052,0.006427895,6.5829685
echo1,0.7299562,0.609260842,0.8745613
age,1.0033034,1.001828522,1.0047805
gender1,1.1435503,0.951370246,1.3745515
weight,0.9900693,0.985987562,0.9941679
elix_score,1.0322733,1.018674448,1.0460537
saps,1.0746388,1.050693041,1.0991303
sofa,1.1880682,1.148672049,1.2288155
vent1,1.1717143,0.863272142,1.5903611
vaso1,1.0401202,0.843537231,1.2825161
